# Tutorial: causal inference methods made easy for applied resarchers/epidemiologists/statisticians 

### ICON-LSHTM, LONDON, 16th October 2020

Miguel Angel Luque Fernandez, PhD

Assistant Professor of Epidemiology and Biostatistics

Matthew Smith, PhD

Research Fellow

Paul Zivich

University of North Carolina at Chapel Hill

Inequalities in Cancer Outcomes Network, LSHTM, London, UK

Copyright (c) 2020 Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NON-INFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Bug reports: miguel-angel.luque at lshtm.ac.uk	

The rhc dataset can be dowloaded at http://biostat.mc.vanderbilt.edu/wiki/Main/DataSets

In [18]:
# Importing libraries for the tutorial
import patsy
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

print("patsy      ", patsy.__version__)
print("numpy      ", np.__version__)
print("pandas     ", pd.__version__)
print("statsmodels", sm.__version__)

patsy       0.5.1
numpy       1.19.1
pandas      1.1.0
statsmodels 0.11.1


## Setting up the Data

In [8]:
data = pd.read_csv("rhc.csv")
print(data.columns)
data.rename(columns={"rhc": "A", 
                     "death_d30": "Y",
                     "gender": "C",
                     "age": "W1",
                     "edu": "W2",
                     "race": "W3",
                     "carcinoma": "W4",
            }, inplace=True)
data = data[["Y", "A", "C", "W1", "W2", "W3", "W4"]].copy()

Index(['Unnamed: 0', 'cat1', 'cat2', 'ca', 'sadmdte', 'dschdte', 'dthdte',
       'lstctdte', 'death', 'cardiohx', 'chfhx', 'dementhx', 'psychhx',
       'chrpulhx', 'renalhx', 'liverhx', 'gibledhx', 'malighx', 'immunhx',
       'transhx', 'amihx', 'age', 'sex', 'edu', 'surv2md1', 'das2d3pc',
       't3d30', 'dth30', 'aps1', 'scoma1', 'meanbp1', 'wblc1', 'hrt1', 'resp1',
       'temp1', 'pafi1', 'alb1', 'hema1', 'bili1', 'crea1', 'sod1', 'pot1',
       'paco21', 'ph1', 'swang1', 'wtkilo1', 'dnr1', 'ninsclas', 'resp',
       'card', 'neuro', 'gastr', 'renal', 'meta', 'hema', 'seps', 'trauma',
       'ortho', 'adld3p', 'urin1', 'race', 'income', 'ptid'],
      dtype='object')


KeyError: "['C', 'W4', 'A', 'Y'] not in index"

## Naive estimate of the ATE

In [20]:
smf.ols("Y ~ A + C", data)
smf.summary()  # ATE = 0.0

TypeError: from_formula() missing 2 required positional arguments: 'formula' and 'data'

## 3. G-Formula

### 3.1 Non-parametric g-formula

In [ ]:
# Non-parametric g-formula for the ATE
np.mean(data.loc[data['C'] == 1, 'A'])
np.mean(data.loc[data['C'] == 0, 'A'])


### 3.2 Parametric g-formula

## 4. Inverse Probability of Treatment Weighting

### 4.1 Inverse probability of treatment weighting based on the propensity score plus regression adjustment

### 4.2 Marginal structural model with stabilised weights

### 4.3 IPTW with regression adjustment

## 5. Augmented Inverse Probability Weighting

## 6. Data-Adaptive Estimation: Ensemble Learning Targeted Maximum Likelihood Estimation

## 7. Simulation

END